In [ ]:
from pathlib import Path
import pandas as pd
import sys
src_path = str(Path.cwd().parent)
if src_path not in sys.path:
    sys.path.append(src_path)

from src.d00_utils.utilities import generate_df_with_imgpaths
from src.d00_utils.dirnames import raw_ometif_dirname, init_rescale_dirname, bg_sub_dirname, user_input_dirname, polygon_ROI_dirname, excluded_dirname

In [ ]:
input_dirpath = input('Please enter the path for the folder containing images')

In [ ]:
def new_or_overwrite_ok(filepath):
    if filepath.exists():
        overwrite_yn = input(f'\'{filepath}\' already exists. Overwrite? Enter Y/N\n')
        if overwrite_yn.lower() == 'n':
            return False
    return True
    

In [ ]:
def create_exclusion_df(proc_dir):
    proc_dir = Path(proc_dir)
    raw_ometif_dirpath = Path(proc_dir) / raw_ometif_dirname
    exclusion_df = generate_df_with_imgpaths(raw_ometif_dirpath)
    exclusion_df['Exclude (Y/N)'] = ''
    exclusion_df['Reason for exclusion'] = ''
    save_dirpath = proc_dir / user_input_dirname
    save_dirpath.mkdir(parents=True, exist_ok=True)
    
    exclusion_list_path = save_dirpath / 'exclusion_list.csv'
    if new_or_overwrite_ok(exclusion_list_path):
        exclusion_df.to_csv(save_dirpath / 'exclusion_list.csv', index=True, index_label='Index')
        print(f'Exclusion list saved to \'{filepath}\'')
    else:
        print('Exclusion list not saved to avoid overwriting existing file.')

In [ ]:
proc_dir = Path(proc_dir)
create_exclusion_df(proc_dir)

In [ ]:
def exclude_images(proc_dir, dirnames='all'):
    
    # Get a list of all directory names in the processing folder'
    if dirnames=='all':
        dirnames = [dirname.name for dirname in proc_dir.iterdir() if (dirname.is_dir and not dirname.name.startswith('.'))]
        print(dirnames)
    
    # Find and open the exclusion list csv file
    exclusion_list_path = proc_dir / user_input_dirname / 'exclusion_list.csv'
    exclusion_df = pd.read_csv(exclusion_list_path)
    
    # Get a list of all filenames 'Y' in the 'Exclude' column
    imgnames_exclude = exclusion_df[exclusion_df['Exclude (Y/N)'].str.contains('Y')]['Image name'].values
    
    for dirname in dirnames:
        
        # Create a new directory for excluded images
        exclude_dirpath = proc_dir / dirname / excluded_dirname
        exclude_dirpath.mkdir(parents=True, exist_ok=True)
        
        # Move images into the directory for excluded images
        ne = 0 # counter for excluded images
        for imgname in imgnames_exclude:
            imgpath = proc_dir / dirname / imgname
            exclude_dirpath = proc_dir / dirname / excluded_dirname
            if imgpath.exists():
                imgpath.rename(exclude_dirpath / imgpath.name)
                ne = ne + 1
        print(f'{ne} images excluded within the {dirname} directory')
                

In [ ]:
proc_dir = Path(proc_dir)
exclude_images(proc_dir, dirnames='all')



In [ ]:
dirnames